- 经典的 VAE 是 Amortized Inference
    - 我们学习一个编码器网络 Encoder(x) -> μ, log(σ²)。这个网络是通用的，它可以为任何输入的x快速计算出其对应的隐变量分布参数。这被称为“摊销式”，因为学习编码器的成本被“摊销”到了所有数据点上。
    - 我们学习一个解码器网络 Decoder(z) -> x'.
    - 训练时，我们同时优化编码器和解码器的参数。

### 后验坍缩（posterior collapse）

VAE 的编码器完全忽略了输入数据 x，导致学到的后验分布 q(z|x) 退化成了与输入无关的先验分布 p(z)。这使得隐变量 z 失去了表达数据信息的能力，整个 VAE 模型也因此失效，无法学习到有意义的数据表示。

$$
p(x)=\int p(z)p(x|z)dz.
$$

$$
\log p(x) \geq \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{\text{KL}}(q(z|x)||p(z))
$$


- KL 散度太容易优化了：对于模型来说，要最小化 KL 散度 KL(q(z|x) || p(z))，最简单的“作弊”方法就是让 q(z|x) 完全等于 p(z)。
- 当 KL 散度项在优化中占据主导地位时，模型会发现一条捷径：
    - 编码器 (Encoder) 选择“躺平”：它不再费力去从输入 x 中提取信息。无论输入什么 x，它都输出一个与 x 无关的、固定的标准正态分布。也就是说，q(z|x) 坍塌成了 p(z)。
    - 隐变量 z 变得无用：因为 z 是从一个与 x 无关的分布中采样得到的，所以它不包含任何关于 x 的特定信息。它就是一堆随机噪声。
    - 解码器 (Decoder) 被迫“自力更生”：解码器发现从编码器传来的 z 毫无用处，为了最小化重构损失，它只能学会忽略 z，然后尽力去生成一个“平均”的、最可能出现的输出。比如，如果训练数据是手写数字，解码器可能会一直生成一个模糊的、像所有数字混合体的图像。

### EM-like VAE (非摊销式):

- E-Step (期望步): 固定模型参数（decoder），估计隐变量的分布。
- M-Step (最大化步): 固定隐变量的分布，更新模型参数以最大化期望似然。

- 我们不再有编码器网络。
- 取而代之，我们为训练集中的每一个数据点 x_i，都直接维护和优化其自身的隐变量分布参数 (μ_i, log(σ²_i))。这些参数不再是网络计算出来的，而是直接作为模型的可训练参数存在。
- 我们只学习一个解码器网络 Decoder(z) -> x'.
- 训练过程分为两个交替进行的步骤，类似EM算法：
    - E-step (Inference Step): 固定解码器，为每个数据点 x_i 优化其对应的隐变量参数 (μ_i, log(σ²_i))，使其能最好地解释该数据点。
    - M-step (Learning Step): 固定所有数据点的隐变量参数，用它们来更新解码器网络，使其能更好地从这些隐变量重构出原始数据。

### mnist experiments

1. 模型构成
- config
    - 隐变量参数 (Latent Parameters):
    - 假设我们有N=60000个MNIST训练样本，隐变量维度为d (例如 d=20)。
    - 我们需要一个参数矩阵 Φ，大小为 `N x (2*d)`。每一行 `Φ[i]` 存储了第 i 个训练样本 x_i 对应的 μ_i 和 log(σ²_i)。
        - 在PyTorch中，这可以很方便地用 torch.nn.Embedding(num_embeddings=N, embedding_dim=2*d) 来实现。我们可以把数据点的索引 i 当作输入，它就会返回对应的 (μ_i, log(σ²_i))。
- 解码器 (Decoder):
    - 这是一个标准的神经网络，$p_θ(x|z)$。
    - 输入：一个从隐变量分布 $q_i(z)$ 采样得到的 z (维度为d)。
    - 输出：一个 28x28 的图像。由于MNIST是二值化的，输出层的激活函数通常是Sigmoid，表示每个像素为1的概率。
    - 损失函数：二元交叉熵 (Binary Cross-Entropy)。
    - 模型参数为 θ。
2. 训练目标 (ELBO)
对于单个数据点 x_i，其ELBO为：

$$ 
ELBO_i = E_{z \sim q_i(z)}[\log p_θ(x_i|z)] - D_{KL}(q_i(z) || p(z))
$$

3. EM-like 训练算法

初始化:
- 随机初始化解码器 Decoder 的参数 θ。
- 初始化所有数据点的隐变量参数 Φ。一个好的选择是全部初始化为0，这对应于先验分布 p(z) (即 μ=0, log(σ²)=0 => σ=1)。
